ETL Pipeline for NLP on textual data from CSV

### **ADP Assignment**

In [1]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [2]:
!pip install textblob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
#Installing the required libraries
import sqlite3
import os
import re
import pandas as pd 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
import numpy as np
import matplotlib.pyplot as plt
from textblob import TextBlob

# **Extract**

In [5]:
#Reading the 'Avengers Tweets' dataset
df = pd.read_csv("/content/drive/My Drive/tweets.csv", encoding='mac_roman', index_col=0)
df.head()

,text,favorited,favoriteCount,replyToSN,created,truncated,replyToSID,id,replyToUID,statusSource,screenName,retweetCount,isRetweet,retweeted,longitude,latitude
1,RT @mrvelstan: literally nobody:\r\nme:\r\n\r\...,False,0,NaN,2019-04-23 10:43:30,False,NaN,1120639328034676737,NaN,"<a href=""http://twitter.com/download/android"" ...",DavidAc96,637,True,False,NaN,NaN
2,"RT @agntecarter: iím emotional, sorry!!\r\n\r\...",False,0,NaN,2019-04-23 10:43:30,False,NaN,1120639325199196160,NaN,"<a href=""http://twitter.com/download/iphone"" r...",NRmalaa,302,True,False,NaN,NaN
3,saving these bingo cards for tomorrow \r\n©\r\...,False,0,NaN,2019-04-23 10:43:30,False,NaN,1120639324683292674,NaN,"<a href=""http://twitter.com/download/iphone"" r...",jijitsuu,0,False,False,NaN,NaN
4,RT @HelloBoon: Man these #AvengersEndgame ads ...,False,0,NaN,2019-04-23 10:43:29,False,NaN,1120639323328540672,NaN,"<a href=""http://twitter.com/download/iphone"" r...",SahapunB,23781,True,False,NaN,NaN
5,"RT @Marvel: We salute you, @ChrisEvans! #Capta...",False,0,NaN,2019-04-23 10:43:29,False,NaN,1120639321571074048,NaN,"<a href=""http://twitter.com/download/iphone"" r...",stella22_97,13067,True,False,NaN,NaN


# **Transform**

In [6]:
def clean_tweets(df):

    """
    Takes raw tweets and cleans them
    so we can carry out analysis
    remove stopwords, punctuation,
    lower case, html, emoticons.
    This will be done using Regex
    ? means option so colou?r matches
    both color and colour.
    """

    # Do some text preprocessing
    stopword_list = stopwords.words('english')
    #ps = PorterStemmer()
    df["clean_tweets"] = None
    #df['len'] = None
    for i in range(1,len(df['tweet'])):
        
        # get rid of anything that isnt a letter

        exclusion_list = ['[^a-zA-Z]','rt', 'http', 'co', 'RT']
        exclusions = '|'.join(exclusion_list)
        text = re.sub(exclusions, ' ' , df['tweet'][i])
        #print(text)
        text = text.lower()
        words = text.split()
        words = [word for word in words if not word in stopword_list]
         # only use stem of word
        #words = [ps.stem(word) for word in words]
        df['clean_tweets'][i] = ' '.join(words)


    # Create column with data length
    #df['len'] = np.array([len(tweet) for tweet in data["clean_tweets"]])

    return df

In [7]:
def sentiment(tweet):
    """
    This function calculates sentiment
    on our cleaned tweets.
    Uses textblob to calculate polarity.
    Parameters:
    ----------------
    arg1: takes in a tweet (row of dataframe)
    """

    # need to improce
    try:
        analysis = TextBlob(tweet)
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'
    except Exception as e:
        pass

# **Load**

In [8]:
def create_table(df, table_name, data_types):
  cols = df.columns
  conn = sqlite3.connect('mydb1.db')
  cursor = conn.cursor()
  create_table_query = f"CREATE TABLE if not exists {table_name}("

  for i in range(0, len(tweet_data.columns)):
    create_table_query += "{} {}, ".format(tweet_data.columns[i], data_types[i])
  create_table_query = create_table_query.rstrip(" ,") + ");"
  #print(create_table_query)
  cursor.execute(create_table_query)
  print("\nTable created successfully!")

In [9]:
def import_to_db(df, table_name):

  conn = sqlite3.connect('mydb1.db')
  cursor = conn.cursor()
  # Write the data to a sqlite db table
  df.to_sql(table_name, conn, if_exists='replace', index=False)

  print("\nImported data successfully!\n")

  # Run select sql query
  cursor.execute(f'select * from {table_name}')

  # Fetch all records
  records = cursor.fetchall()

  # Display result 
  for row in records:
    # show row
    print(row)
    
  # Close connection to SQLite database
  conn.close()

## **ETL Pipeline - Driver Code**

In [ ]:
#Extract
tweet_data = df[:100]
tweet_data.rename(columns = {'text':'tweet'}, inplace = True)
tweet_data = tweet_data[['tweet','created', 'id', 'screenName', 'retweetCount', 'isRetweet', 'retweeted']]

#Transform
tweet_data = clean_tweets(tweet_data)
tweet_data['Sentiment'] = np.array([sentiment(x) for x in tweet_data['clean_tweets']])
print(tweet_data)

#Load
table_name = 'TWEETS'
data_types = ['TEXT', 'TEXT', 'INT', 'TEXT', 'INT', 'TEXT', 'TEXT', 'TEXT', 'TEXT']
create_table(tweet_data, table_name, data_types)
import_to_db(tweet_data, table_name)